In [2]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv("G:\\sentiment analysis\\reviews_data_dump\\reviews_badminton\\data.csv")

In [4]:
df.head()

,Reviewer Name,Review Title,Place of Review,Up Votes,Down Votes,Month,Review text,Ratings
0,Kamal Suresh,Nice product,"Certified Buyer, Chirakkal",889.0,64.0,Feb 2021,"Nice product, good quality, but price is now r...",4
1,Flipkart Customer,Don't waste your money,"Certified Buyer, Hyderabad",109.0,6.0,Feb 2021,They didn't supplied Yonex Mavis 350. Outside ...,1
2,A. S. Raja Srinivasan,Did not meet expectations,"Certified Buyer, Dharmapuri",42.0,3.0,Apr 2021,Worst product. Damaged shuttlecocks packed in ...,1
3,Suresh Narayanasamy,Fair,"Certified Buyer, Chennai",25.0,1.0,NaN,"Quite O. K. , but nowadays the quality of the...",3
4,ASHIK P A,Over priced,NaN,147.0,24.0,Apr 2016,Over pricedJust â?¹620 ..from retailer.I didn'...,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8518 entries, 0 to 8517
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Reviewer Name    8508 non-null   object 
 1   Review Title     8508 non-null   object 
 2   Place of Review  8468 non-null   object 
 3   Up Votes         8508 non-null   float64
 4   Down Votes       8508 non-null   float64
 5   Month            8053 non-null   object 
 6   Review text      8510 non-null   object 
 7   Ratings          8518 non-null   int64  
dtypes: float64(2), int64(1), object(5)
memory usage: 532.5+ KB


In [6]:
df.describe()

,Up Votes,Down Votes,Ratings
count,8508.000000,8508.000000,8518.000000
mean,0.391396,0.121768,4.181028
std,11.613909,3.248022,1.262200
min,0.000000,0.000000,1.000000
25%,0.000000,0.000000,4.000000
50%,0.000000,0.000000,5.000000
75%,0.000000,0.000000,5.000000
max,889.000000,219.000000,5.000000


In [7]:
df.isnull().sum()

Reviewer Name       10
Review Title        10
Place of Review     50
Up Votes            10
Down Votes          10
Month              465
Review text          8
Ratings              0
dtype: int64

In [8]:
df.duplicated().sum()

np.int64(4)

In [9]:
df=df.drop_duplicates()

In [10]:
df.duplicated().sum()

np.int64(0)

In [11]:
df.shape

(8514, 8)

In [12]:
df = df[['Review text', 'Ratings']]


In [13]:
df.head()

,Review text,Ratings
0,"Nice product, good quality, but price is now r...",4
1,They didn't supplied Yonex Mavis 350. Outside ...,1
2,Worst product. Damaged shuttlecocks packed in ...,1
3,"Quite O. K. , but nowadays the quality of the...",3
4,Over pricedJust â?¹620 ..from retailer.I didn'...,1


In [14]:
df.isnull().sum()

Review text    4
Ratings        0
dtype: int64

In [15]:
df = df.dropna(subset=['Review text', 'Ratings'])


In [16]:
def sentiment_from_rating(r):
    if r >= 4:
        return 1   
    elif r <= 2:
        return 0  
    else:
        return None  

df['sentiment'] = df['Ratings'].apply(sentiment_from_rating)
df = df.dropna()


In [17]:
df['sentiment'].value_counts()


sentiment
1.0    6823
0.0    1072
Name: count, dtype: int64

In [18]:
import re
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

df['clean_review'] = df['Review text'].apply(clean_text)


In [19]:
from sklearn.model_selection import train_test_split

X = df['clean_review']
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1,2)
)

X_train_vec = tfidf.fit_transform(X_train)
X_test_vec = tfidf.transform(X_test)


In [21]:
from sklearn.svm import LinearSVC

model = LinearSVC(C=1.0)
model.fit(X_train_vec, y_train)


LinearSVC()

In [22]:
from sklearn.metrics import classification_report, f1_score

y_pred = model.predict(X_test_vec)

print(classification_report(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))


              precision    recall  f1-score   support

         0.0       0.78      0.62      0.69       214
         1.0       0.94      0.97      0.96      1365

    accuracy                           0.92      1579
   macro avg       0.86      0.79      0.82      1579
weighted avg       0.92      0.92      0.92      1579

F1 Score: 0.9571171171171171


In [23]:
import joblib

joblib.dump(model, "flipkart_sentiment_model.pkl")
joblib.dump(tfidf, "tfidf_vectorizer.pkl")


['tfidf_vectorizer.pkl']

In [24]:
joblib.dump(model, "flipkart_sentiment_model.pkl")

['flipkart_sentiment_model.pkl']

In [25]:
print(joblib.__version__)

1.4.2


In [28]:
import sklearn
print(sklearn.__version__)

1.6.1
